# day 24

https://adventofcode.com/2019/day/24

In [1]:
import logging
import logging.config
import os

import yaml

In [2]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [3]:
FNAME = os.path.join('data', 'day24.txt')

LOGGER = logging.getLogger('day24')

## part 1

### problem statement:

#### loading data

In [7]:
test_data = """....#
#..#.
#..##
..#..
#...."""
test_answer = 2129920

In [8]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [10]:
print(load_data())

#..##
#.#..
#...#
##..#
#..##


#### function def

In [25]:
SPACE = 0
BUG = 1

SPACE_CHAR = '.'
BUG_CHAR = '#'

def parse_map(s):
    return {(i + 1j * j): SPACE if c == SPACE_CHAR else BUG
            for (i, row) in enumerate(s.split('\n'))
            for (j, c) in enumerate(row)}

In [42]:
def step(g):
    new_g = {}
    for (k, v) in g.items():
        num_bug_neighbors = sum(g.get(k + delta, SPACE) == BUG
                                for delta in [+1, -1, +1j, -1j])
        if v == BUG and num_bug_neighbors != 1:
            new_g[k] = SPACE
        elif v == SPACE and num_bug_neighbors in [1, 2]:
            new_g[k] = BUG
        else:
            new_g[k] = v
    return new_g

In [43]:
def map_to_str(g):
    s = ''
    for i in range(5):
        for j in range(5):
            k = i + (1j * j)
            s += BUG_CHAR if g[k] == BUG else SPACE_CHAR
        s += '\n'
    return s

In [44]:
g = parse_map(test_data)
print(map_to_str(g))

....#
#..#.
#..##
..#..
#....



In [45]:
g = step(g)
print(map_to_str(g))

#..#.
####.
###.#
##.##
.##..



In [49]:
def hashable(g):
    return tuple(g.items())

In [54]:
def q_1(data):
    g = parse_map(data)
    seen_grids = set(hashable(g))
    while True:
        g = step(g)
        hg = hashable(g)
        if hg in seen_grids:
            return sum(2 ** (5 * k.real + k.imag)
                       for (k, v) in g.items()
                       if v == BUG)
        else:
            seen_grids.add(hg)

#### tests

In [55]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == test_answer
    LOGGER.setLevel(logging.INFO)

In [56]:
test_q_1()

#### answer

In [57]:
q_1(load_data())

30446641.0

## part 2

### problem statement:

#### function def

In [78]:
from collections import defaultdict, namedtuple

GridLoc = namedtuple('GridLoc', ['ij', 'level'])

In [82]:
import functools

In [109]:
UP = -1
DOWN = 1
LEFT = -1j
RIGHT = 1j

@functools.lru_cache()
def _smart(gl):
    LOGGER.info(f'gl.ij = {gl.ij}')
    # the four outer walls
    if gl.ij.real < 0:
        return [GridLoc(1 + 2j, gl.level + 1)]
    elif gl.ij.real > 4:
        return [GridLoc(3 + 2j, gl.level + 1)]
    elif gl.ij.imag < 0:
        return [GridLoc(2 + 1j, gl.level + 1)]
    elif gl.ij.imag > 4:
        return [GridLoc(2 + 3j, gl.level + 1)]
    # the four inner walls
    elif gl.ij == 1 + 2j:
        return [GridLoc(0 + 1j * j, gl.level - 1)
                for j in range(5)]
    elif gl.ij == 3 + 2j:
        return [GridLoc(4 + 1j * j, gl.level - 1)
                for j in range(5)]
    elif gl.ij == 2 + 1j:
        return [GridLoc(i, gl.level - 1)
                for i in range(5)]
    elif gl.ij == 2 + 3j:
        return [GridLoc(i + 4j, gl.level - 1)
                for i in range(5)]
    else:
        return [gl]

@functools.lru_cache()
def get_neighbors(gl):
    return [nbr
            for direction in [UP, DOWN, LEFT, RIGHT]
            for nbr in _smart(GridLoc(gl.ij + direction, gl.level))]

In [110]:
get_neighbors(GridLoc(3 + 3j, 1))

2019-12-24 00:58:04,653 INFO     [day24._smart:8] gl.ij = (2+3j)
2019-12-24 00:58:04,654 INFO     [day24._smart:8] gl.ij = (4+3j)
2019-12-24 00:58:04,655 INFO     [day24._smart:8] gl.ij = (3+2j)
2019-12-24 00:58:04,657 INFO     [day24._smart:8] gl.ij = (3+4j)


[GridLoc(ij=4j, level=0),
 GridLoc(ij=(1+4j), level=0),
 GridLoc(ij=(2+4j), level=0),
 GridLoc(ij=(3+4j), level=0),
 GridLoc(ij=(4+4j), level=0),
 GridLoc(ij=(4+3j), level=1),
 GridLoc(ij=(4+0j), level=0),
 GridLoc(ij=(4+1j), level=0),
 GridLoc(ij=(4+2j), level=0),
 GridLoc(ij=(4+3j), level=0),
 GridLoc(ij=(4+4j), level=0),
 GridLoc(ij=(3+4j), level=1)]

In [102]:
CENTER_IJ = 2 + 2j

def q_2(data):
    grid = defaultdict(int)
    
    # parse the starting grid
    for ij, c in parse_map(data).items():
        if ij != CENTER_IJ:
            grid[ij] = c

    return grid

In [73]:
grid = q_2(load_data())

In [74]:
grid

defaultdict(int,
            {0j: 1,
             1j: 0,
             2j: 0,
             3j: 1,
             4j: 1,
             (1+0j): 1,
             (1+1j): 0,
             (1+2j): 1,
             (1+3j): 0,
             (1+4j): 0,
             (2+0j): 1,
             (2+1j): 0,
             (2+3j): 0,
             (2+4j): 1,
             (3+0j): 1,
             (3+1j): 1,
             (3+2j): 0,
             (3+3j): 0,
             (3+4j): 1,
             (4+0j): 1,
             (4+1j): 0,
             (4+2j): 0,
             (4+3j): 1,
             (4+4j): 1})

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin